In [1]:
import glob
import os
import pandas as pd
from datetime import datetime, timedelta 
from dateutil.relativedelta import relativedelta
from src.func_data_engineering import *

# auto load modules
%load_ext autoreload
%autoreload

%cd C:\Users\tyler\repos\airbnb-forecast

C:\Users\tyler\repos\airbnb-forecast


In [2]:
# store raw data
# listing

cols = ['id', 'last_scraped', 'host_id', 'host_since','host_neighbourhood',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'calculated_host_listings_count',
     'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms',
 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 
         'street',
      'neighbourhood_cleansed','city', 'state', 'zipcode', 'market',
       'smart_location','country', 'latitude', 'longitude',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', 'square_feet',
       'price', 'weekly_price', 'monthly_price', 'security_deposit',
       'cleaning_fee', 'guests_included', 'extra_people', 'minimum_nights',
       'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights',
       'minimum_maximum_nights', 'maximum_maximum_nights',
       'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'calendar_updated',
       'has_availability', 'availability_30', 'availability_60',
       'availability_90', 'availability_365', 'calendar_last_scraped',
       'number_of_reviews', 'number_of_reviews_ltm', 'first_review',
       'last_review', 'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value','instant_bookable', 'is_business_travel_ready',
       'cancellation_policy', 'require_guest_profile_picture',
       'require_guest_phone_verification', 
        'reviews_per_month']

path = r'data\20'
all_files = glob.glob(os.path.join(path,"listings*.csv"))

ls_file = []
for file in all_files:
    df = pd.read_csv(file,header=0)
    df = df[cols]
    ls_file.append(df)

df_list = pd.concat(ls_file,axis=0,ignore_index=True)

C:\Users\tyler\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (43,61,62,94) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\tyler\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (61,62,94) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_list.columns.str.upper()

Index(['ID', 'LAST_SCRAPED', 'HOST_ID', 'HOST_SINCE', 'HOST_NEIGHBOURHOOD',
       'HOST_RESPONSE_TIME', 'HOST_RESPONSE_RATE', 'HOST_ACCEPTANCE_RATE',
       'HOST_IS_SUPERHOST', 'CALCULATED_HOST_LISTINGS_COUNT',
       'CALCULATED_HOST_LISTINGS_COUNT_ENTIRE_HOMES',
       'CALCULATED_HOST_LISTINGS_COUNT_PRIVATE_ROOMS',
       'CALCULATED_HOST_LISTINGS_COUNT_SHARED_ROOMS', 'HOST_VERIFICATIONS',
       'HOST_HAS_PROFILE_PIC', 'HOST_IDENTITY_VERIFIED', 'STREET',
       'NEIGHBOURHOOD_CLEANSED', 'CITY', 'STATE', 'ZIPCODE', 'MARKET',
       'SMART_LOCATION', 'COUNTRY', 'LATITUDE', 'LONGITUDE',
       'IS_LOCATION_EXACT', 'PROPERTY_TYPE', 'ROOM_TYPE', 'ACCOMMODATES',
       'BATHROOMS', 'BEDROOMS', 'BEDS', 'BED_TYPE', 'AMENITIES', 'SQUARE_FEET',
       'PRICE', 'WEEKLY_PRICE', 'MONTHLY_PRICE', 'SECURITY_DEPOSIT',
       'CLEANING_FEE', 'GUESTS_INCLUDED', 'EXTRA_PEOPLE', 'MINIMUM_NIGHTS',
       'MAXIMUM_NIGHTS', 'MINIMUM_MINIMUM_NIGHTS', 'MAXIMUM_MINIMUM_NIGHTS',
       'MINIMUM_MAXIMUM_NIG

In [63]:
df_list.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148835 entries, 0 to 148834
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   latitude   148835 non-null  float64
 1   longitude  148835 non-null  float64
dtypes: float64(2)
memory usage: 2.3 MB


In [ ]:
# upload FS_LIST_RAW

fs_upload(engine,df_list,'FS_LIST_RAW')

In [ ]:
# features for model

col_list = ['id', 'last_scraped', 'host_since','host_neighbourhood',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'calculated_host_listings_count',
         'host_verifications','host_has_profile_pic', 'host_identity_verified', 
         'street','neighbourhood_cleansed',
        'city', 'state', 'zipcode', 'market', 'country', 'latitude', 'longitude',
       'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'bed_type', 
       'price', 'security_deposit',
       'cleaning_fee', 'guests_included', 'extra_people', 'minimum_nights',
       'maximum_nights', 'calendar_updated',
       'number_of_reviews','number_of_reviews_ltm', 'first_review',
       'last_review', 'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value','instant_bookable', 'is_business_travel_ready',
       'cancellation_policy', 'require_guest_profile_picture',
       'require_guest_phone_verification', 
        'reviews_per_month']

In [41]:
# calendar data

# read files
all_cal= glob.glob(os.path.join(path,"calendar*.csv"))

ls_cal = []
for file in all_cal:
    df = pd.read_csv(file,header=0)
    #df['SCRAPED_DATE'] = df.date.min()
    ls_cal.append(df)

    
# for each monthly scrape, only keep the calendar data b/w the scraped date and the date of next scrape run
ls_month = []
num_cal = len(ls_cal)
for i in range(num_cal):
    if i < num_cal-1:
        date_end = ls_cal[i+1].date.min()
        df = ls_cal[i]
        df = df[df.date < date_end]
    else:
        df = ls_cal[i]
        date_start = pd.to_datetime(df.date.min()).date()
        date_end = date_start+relativedelta(months=1)
        df = df[df.date<str(date_end)]
    ls_month.append(df)

df_cal = pd.concat(ls_month,axis=0,ignore_index=True)


# clean
df_cal.columns = df_cal.columns.str.upper()
df_cal['ADJUSTED_PRICE'] = df_cal['ADJUSTED_PRICE'].str.replace('$','')
df_cal['ADJUSTED_PRICE'] = df_cal['ADJUSTED_PRICE'].str.replace(',','')
df_cal['ADJUSTED_PRICE']=df_cal['ADJUSTED_PRICE'].astype(float) # convert to float to keep nan values
df_cal['PRICE'] = df_cal['PRICE'].str.replace('$','')
df_cal['PRICE'] = df_cal['PRICE'].str.replace(',','')
df_cal['PRICE']=df_cal['PRICE'].astype(float)
df_cal['BOOKED'] = df_cal.AVAILABLE.replace({'t':0,'f':1})

In [29]:
df_cal.head()

,LISTING_ID,DATE,AVAILABLE,PRICE,ADJUSTED_PRICE,MINIMUM_NIGHTS,MAXIMUM_NIGHTS,BOOKED
0,251965,2020-01-07,f,259.0,259.0,1.0,365.0,1
1,653384,2020-01-07,f,100.0,100.0,1.0,1125.0,1
2,653384,2020-01-08,f,100.0,100.0,1.0,1125.0,1
3,653384,2020-01-09,f,100.0,100.0,1.0,1125.0,1
4,653384,2020-01-10,f,100.0,100.0,1.0,1125.0,1


In [43]:
df_cal.shape

(4555609, 9)

In [28]:
# upload FS_CAL_RAW

engine = connect_my_db('secrets/db_string')
fs_upload(engine,df_cal,'FS_CAL_RAW')

1143.5180366039276

In [48]:
# aggregate to monthly

df_cal['YEAR_MONTH'] = df_cal.DATE.str[:7]

df_month = df_cal[['LISTING_ID','YEAR_MONTH','ADJUSTED_PRICE','BOOKED']].groupby(['LISTING_ID','YEAR_MONTH']).\
agg({'ADJUSTED_PRICE':'median','BOOKED':'sum'}).reset_index()

# drop the half last month
df_month=df_month[~(df_month.YEAR_MONTH=='2020-07')]

In [56]:
df_month

,LISTING_ID,YEAR_MONTH,ADJUSTED_PRICE,BOOKED
0,9835,2020-01,60.0,0
1,9835,2020-02,60.0,0
2,9835,2020-03,60.0,0
3,9835,2020-04,60.0,0
4,9835,2020-05,60.0,0
...,...,...,...,...
180369,43733191,2020-06,320.0,11
180371,43744200,2020-06,55.0,15
180373,43746933,2020-06,140.0,7
180375,43748923,2020-06,79.0,19


In [ ]:
# upload FS_CAL_MONTHLY

fs_upload(engine,df_cal,'FS_CAL_MONTHLY')

In [ ]:
fre=cal.listing_id.value_counts()

In [ ]:
cal.available=cal.available.replace({'t':1,'f':0})
cal.head()

In [ ]:
cal.date.max()

In [ ]:
cal[['listing_id','available']].groupby(['listing_id']).sum()

In [ ]:
fre.unique()

In [6]:
fre.unique()

In [9]:
cal.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,323280,2020-06-12,t,$67.00,$67.00,1.0,730365.0
1,315513,2020-06-12,f,$45.00,$45.00,1.0,1125.0
2,315513,2020-06-13,f,$45.00,$45.00,1.0,1125.0
3,315513,2020-06-14,f,$45.00,$45.00,1.0,1125.0
4,315513,2020-06-15,f,$45.00,$45.00,1.0,1125.0


In [10]:
fre=cal.listing_id.value_counts()

In [19]:
cal.available=cal.available.replace({'t':1,'f':0})
cal.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,323280,2020-06-12,1,$67.00,$67.00,1.0,730365.0
1,315513,2020-06-12,0,$45.00,$45.00,1.0,1125.0
2,315513,2020-06-13,0,$45.00,$45.00,1.0,1125.0
3,315513,2020-06-14,0,$45.00,$45.00,1.0,1125.0
4,315513,2020-06-15,0,$45.00,$45.00,1.0,1125.0


In [21]:
cal.date.max()

'2021-06-13'

In [20]:
cal[['listing_id','available']].groupby(['listing_id']).sum()

,available
listing_id,
9835,365
10803,41
12936,0
38271,351
41836,0
...,...
43733191,79
43744200,4
43746933,173


In [15]:
fre.unique()

array([367, 366, 365], dtype=int64)